In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
from sklearn import preprocessing
import netron
from keras.models import load_model
import pickle
import os
import csv
from csv import writer
from csv import reader

In [ ]:
#set the header list
header_list = ["Frame","Face","keypoint","Body","1_x","1_y","1_accuracy","2_x","2_y","2_accuracy","3_x","3_y","3_accuracy","4_x","4_y","4_accuracy","5_x","5_y","5_accuracy","6_x","6_y","6_accuracy","7_x","7_y","7_accuracy","8_x","8_y","8_accuracy","9_x","9_y","9_accuracy",
               "10_x","10_y","10_accuracy","11_x","11_y","11_accuracy","12_x","12_y","12_accuracy","13_x","13_y","13_accuracy","14_x","14_y","14_accuracy","15_x","15_y","15_accuracy","16_x","16_y","16_accuracy","17_x","17_y","17_accuracy","18_x","18_y","18_accuracy","19_x","19_y","19_accuracy",
               "20_x","20_y","20_accuracy","21_x","21_y","21_accuracy","22_x","22_y","22_accuracy","23_x","23_y","23_accuracy","24_x","24_y","24_accuracy","25_x","25_y","25_accuracy"]
df = pd.read_csv('merged_json_csv_test.csv', names=header_list)
#set lists with frame data extracted with dlib and openpose
frame=df['Frame']
face=df['Face']
body=df['Body']

le = preprocessing.LabelEncoder()
# Converting string labels into numbers.
face_encoded=le.fit_transform(face)
body_encoded=le.fit_transform(body)

print ("Frame:", frame)
print ("Face:", face_encoded)
print("Body:", body_encoded)
#create the list with images
test_image = []
for i in tqdm(range(frame.shape[0])):
    img = image.load_img('C:\\Users\\CASALAB\\Desktop\\progetto_CASA\\prova3/'+frame[i][:6]+'/'+frame[i][-4:]+'.jpg',target_size=(150,150,3))
    img = image.img_to_array(img)
    img = img/255
    test_image.append(img)

In [ ]:
#load the model with 1 dense layer and 25 train epoch
modelA = load_model('ModelloAllenato_shuffle.hdf5')

In [ ]:
test_body=[]
test_face=[]
#made the prediction about th eentire image array
proba = modelA.predict(np.array(test_image))
#label the model's output
for x in range(len(test_image)):
    proba[x][1]=proba[x][1]
    test_face.append(int(bool(proba[x][0]>0.5)))
    test_body.append(int(bool(proba[x][1]>0.5)))
#count the right respons
right=0
right_face=0
right_body=0
for x in range(len(test_body)):
    if test_face[x]==face_encoded[x] and test_body[x]==body_encoded[x]:
        right+=1
for x in range(len(test_body)):
    if test_body[x]==body_encoded[x]:
        right_body+=1
for x in range(len(test_face)):
    if test_face[x]==face_encoded[x]:
        right_face+=1
        
#write the output csv with correct respons and prediction
with open('test2.csv', mode='w', newline='') as jsonfile:
    json_writer = csv.writer(jsonfile)
    for x in range(len(test_body)):
        json_writer.writerow([face_encoded[x]]+[body_encoded[x]]+[test_face[x]]+[test_body[x]])
        
#calculate and print the precision 
accuracy=right*100/len(test_body)
print("Accuracy:"+str(accuracy)+"%")
face_accuracy=right_face*100/len(test_face)
print("Face Accuracy:"+str(face_accuracy)+"%")
body_accuracy=right_body*100/len(test_body)
print("Body Accuracy:"+str(body_accuracy)+"%")
#print the output csv file
df = pd.read_csv('test.csv', names=['face','body','test_face','test_body'])
df
    